In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
train = pd.read_hdf('data/train.h5')
excl = ['id', 'sample', 'y', 'timestamp']
col = [c for c in train.columns if c not in excl]

In [ ]:
train.head()

In [ ]:
train.set_index('timestamp')['fundamental_0'].plot()